# Orientation Estimation


## Reference

- https://towardsdatascience.com/matplotlib-animations-in-jupyter-notebook-4422e4f0e389
    
- https://matplotlib.org/stable/gallery/animation/simple_anim.html 

- https://seanng.my/2020/07/15/wireless-imu-sensor-using-bluetooth-low-energy-ble/

- https://roboticsclubiitk.github.io/2017/12/21/Beginners-Guide-to-IMU.html

- Complementary filter https://www.pieter-jan.com/node/11

- Arduino IMUhttps://docs.arduino.cc/tutorials/nano-33-ble-sense/imu_accelerometer

- https://stackoverflow.com/questions/23009549/roll-pitch-yaw-calculation

- https://habr.com/en/post/499190/

 - https://docs.arduino.cc/tutorials/nano-33-ble-sense/imu_magnetometer
 
 - https://docs.arduino.cc/tutorials/nano-33-ble-sense/imu_gyroscope
 
- https://www.infineon.com/dgdl/Infineon-3D_Magnetic_Sensor_for_Angle_Measurements-AN-v01_10-AN-v01_01-EN.pdf?fileId=5546d46265f064ff01665a3d22f055e3

![style=centerme](....png)

In [7]:
import matplotlib.pylab as plt
import numpy as np
import matplotlib.animation as animation
import pandas as pd
import time 
import collections
import serial
import threading

In [8]:
%pylab
#%matplotlib inline

x = np.linspace(0, 20, 1000)  # 100 evenly-spaced values from 0 to 50
y = np.sin(x)

plt.plot(x, y)

Using matplotlib backend: MacOSX
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/Users/Tommaso/miniconda3/lib/python3.9/site-packages/IPython/core/magics/pylab.py:162: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


Traceback (most recent call last):
  File "/Users/Tommaso/miniconda3/lib/python3.9/site-packages/matplotlib/cbook/__init__.py", line 287, in process
    func(*args, **kwargs)
  File "/Users/Tommaso/miniconda3/lib/python3.9/site-packages/matplotlib/animation.py", line 913, in _start
    self.event_source.add_callback(self._step)
AttributeError: 'NoneType' object has no attribute 'add_callback'


In [9]:
import numpy as np
#collections.deque(np.arange(1000), maxlen=1000),

In [10]:
class arduino(object):#serial_port,baud_rate=baud_rate):
    
    def __init__(self,serial_port='/dev/cu.usbmodem14201',baud_rate=9600,vector_lenght=1000,alpha=0.02):
        
        self.serial_port   = serial_port
        self.baud_rate     = baud_rate
        
        self.alpha         = 0.02
        
        self.vector_lenght = vector_lenght
        
        self.Timestamp    = collections.deque(np.arange(vector_lenght), maxlen=vector_lenght)
        self.TimeInterval = collections.deque(np.arange(vector_lenght), maxlen=vector_lenght)
        
        self.Accelerometer = {'x':collections.deque([0]*vector_lenght, maxlen=vector_lenght),
                              'y':collections.deque([0]*vector_lenght, maxlen=vector_lenght),
                              'z':collections.deque([0]*vector_lenght, maxlen=vector_lenght)}
        self.Gyroscope     = {'x':collections.deque([0]*vector_lenght, maxlen=vector_lenght),
                              'y':collections.deque([0]*vector_lenght, maxlen=vector_lenght),
                              'z':collections.deque([0]*vector_lenght, maxlen=vector_lenght)}
        self.Magnetometer  = {'x':collections.deque([0]*vector_lenght, maxlen=vector_lenght),
                              'y':collections.deque([0]*vector_lenght, maxlen=vector_lenght),
                              'z':collections.deque([0]*vector_lenght, maxlen=vector_lenght)}
        
        self.Pitch = {'Accelerometer':collections.deque([0]*vector_lenght, maxlen=vector_lenght),
                      'Gyroscope'    :collections.deque([0]*vector_lenght, maxlen=vector_lenght),
                      'Magnetometer' :collections.deque([0]*vector_lenght, maxlen=vector_lenght),
                      'Complementary':collections.deque([0]*vector_lenght, maxlen=vector_lenght)}
        self.Yaw = {'Accelerometer':collections.deque([0]*vector_lenght, maxlen=vector_lenght),
                      'Gyroscope'    :collections.deque([0]*vector_lenght, maxlen=vector_lenght),
                      'Magnetometer' :collections.deque([0]*vector_lenght, maxlen=vector_lenght),
                      'Complementary':collections.deque([0]*vector_lenght, maxlen=vector_lenght)}
        self.Roll   = {'Accelerometer':collections.deque([0]*vector_lenght, maxlen=vector_lenght),
                      'Gyroscope'    :collections.deque([0]*vector_lenght, maxlen=vector_lenght),
                      'Magnetometer' :collections.deque([0]*vector_lenght, maxlen=vector_lenght),
                      'Complementary':collections.deque([0]*vector_lenght, maxlen=vector_lenght)}

    def read_serial_monitor(self):
    
        arduinoString = self.arduinoData.readline()
        arduinoString = arduinoString.decode("utf-8")
        arduinoString = arduinoString.strip()

        self.char_list     = arduinoString.split('\t')  
        
        #print("\r{:}".format(self.char_list),end='')
        #print("\rtest",end='')
        
        
    def wait_serial_monitor(self):
        
        self.read_serial_monitor()
        while len(self.char_list)!=10: 
            print("\rConnecting...",end='')
            self.read_serial_monitor()
        print("\rConnected     ",end='')


    def get_new_data(self):
    
        self.read_serial_monitor()
        
        timestamp,ax,ay,az,gx,gy,gz,mx,my,mz = self.char_list
        
        if mz == 'ovf': 
            mz = 0
            
        timestamp,ax,ay,az,gx,gy,gz,mx,my,mz = float(timestamp),float(ax),float(ay),float(az),float(gx),float(gy),float(gz),float(mx),float(my),float(mz)

        timestamp /= 1000
        gx,gy,gz  = np.deg2rad(gx),np.deg2rad(gy),np.deg2rad(gz)
        
        self.update_raw_vectors(timestamp,ax,ay,az,gx,gy,gz,mx,my,mz)
        
    def update_raw_vectors(self,timestamp,ax,ay,az,gx,gy,gz,mx,my,mz):
        
        self.Timestamp.append(float(timestamp))
        self.TimeInterval.append(self.Timestamp[-1]-self.Timestamp[-2])
        self.Accelerometer['x'].append(ax);self.Accelerometer['y'].append(ay);self.Accelerometer['z'].append(az)
        self.Gyroscope['x'].append(gx);    self.Gyroscope['y'].append(gy);    self.Gyroscope['z'].append(gz)
        self.Magnetometer['x'].append(mx); self.Magnetometer['y'].append(my); self.Magnetometer['z'].append(mz)

   
    def read(self):
            
        self.arduinoData = serial.Serial(self.serial_port, self.baud_rate) 
        
        self.wait_serial_monitor()
        self.get_new_data()
        
        while True:
            
            self.get_new_data()
            
            alpha = 0.02
            self.Pitch['Accelerometer'].append( np.arctan2(self.Accelerometer['x'][-1],np.sqrt(self.Accelerometer['y'][-1]**2+self.Accelerometer['z'][-1]**2)))
            self.Pitch['Gyroscope'].append(self.Gyroscope['y'][-1]*self.TimeInterval[-1]+self.Pitch['Gyroscope'][-1])
            #self.Pitch['Magnetometer']
            self.Pitch['Complementary'].append((1-alpha)*(self.Pitch['Complementary'][-1]  + self.Gyroscope['y'][-1]*self.TimeInterval[-1]) + alpha*self.Pitch['Accelerometer'][-1])
            
            self.Roll['Accelerometer'].append( np.arctan2(-self.Accelerometer['y'][-1],self.Accelerometer['z'][-1]))
            self.Roll['Gyroscope'].append(self.Gyroscope['x'][-1]*self.TimeInterval[-1]+self.Roll['Gyroscope'][-1])
            #self.Roll['Magnetometer']
            self.Roll['Complementary'].append((1-alpha)*(self.Roll['Complementary'][-1]  + self.Gyroscope['x'][-1]*self.TimeInterval[-1]) + alpha*self.Roll['Accelerometer'][-1])
            
            if (self.Magnetometer['x'][-1]==0)&(self.Magnetometer['y'][-1]==0)&(self.Magnetometer['z'][-1]==0):
                
                self.Magnetometer['x'][-1],self.Magnetometer['y'][-1],self.Magnetometer['z'][-1]=self.Magnetometer['x'][-2],self.Magnetometer['y'][-2],self.Magnetometer['z'][-2]
                #mx,my,mz = mx-11.18,my-22.15,mz+5.35
            
            #self.Yaw['Accelerometer']
            self.Yaw['Gyroscope'].append(self.Gyroscope['z'][-1]*self.TimeInterval[-1]+self.Yaw['Gyroscope'][-1])
            mag_x = self.Magnetometer['x'][-1] * np.cos(self.Pitch['Complementary'][-1]) + self.Magnetometer['y'][-1] * np.sin(self.Roll['Complementary'][-1]) * np.sin(self.Pitch['Complementary'][-1]) + self.Magnetometer['z'][-1] * np.cos(self.Roll['Complementary'][-1]) * np.sin(self.Pitch['Complementary'][-1])
            mag_y = self.Magnetometer['y'][-1] * np.cos(self.Roll['Complementary'][-1])  - self.Magnetometer['z'][-1] * np.sin(self.Roll['Complementary'][-1])
            self.Yaw['Magnetometer'].append(np.arctan2(- mag_y,mag_x));
            self.Yaw['Complementary'].append((1-alpha)*(self.Yaw['Complementary'][-1]  + self.Gyroscope['z'][-1]*self.TimeInterval[-1]) + alpha*self.Yaw['Magnetometer'][-1])
            
            #print("\r{:100.5f}".format(self.Accelerometer['x'][-1]),end='')
            #print("\r{:100.0f}".format(self.Timestamp[-1]),end='')
            
            

# Single plot

In [6]:
my_arduino = arduino()
#Loops the in_background function
try:
    thread = threading.Thread(target = my_arduino.read)
    thread.start() 

except KeyboardInterrupt:
    thread._Thread_stop() 
    #sys.exit()
    
fig, ax = plt.subplots(1,1)

line,  = ax.plot(my_arduino.Timestamp,my_arduino.Accelerometer['x']);ax.set_ylim([-1.5,1.5]);ax.grid(True)

def animate(i):
    
    line.set_ydata(my_arduino.Accelerometer['x'])

ani = animation.FuncAnimation(fig, animate, interval=200)

Connected     

# Multiple plot

In [12]:
my_arduino = arduino()
#Loops the in_background function
try:
    thread = threading.Thread(target = my_arduino.read)
    thread.start() 

except KeyboardInterrupt:
    thread._Thread_stop() 
    #sys.exit()
    
fig, (ax1,ax2,ax3) = plt.subplots(3,1,figsize=[6.4,4.8*3])

Ax, = ax1.plot(np.arange(1000),my_arduino.Accelerometer['x'])
Ay, = ax1.plot(np.arange(1000),my_arduino.Accelerometer['y'])
Az, = ax1.plot(np.arange(1000),my_arduino.Accelerometer['z'])

Gx, = ax2.plot(np.arange(1000),my_arduino.Gyroscope['x'])
Gy, = ax2.plot(np.arange(1000),my_arduino.Gyroscope['y'])
Gz, = ax2.plot(np.arange(1000),my_arduino.Gyroscope['z'])

Mx, = ax3.plot(np.arange(1000),my_arduino.Magnetometer['x'])
My, = ax3.plot(np.arange(1000),my_arduino.Magnetometer['y'])
Mz, = ax3.plot(np.arange(1000),my_arduino.Magnetometer['z'])

ax1.grid(True);ax2.grid(True);ax3.grid(True)


ax1.set_ylim([-1.5,1.5]);ax2.set_ylim([-25,25]);ax3.set_ylim([-50,50])

def animate(i):
    
    Ax.set_ydata(my_arduino.Accelerometer['x'])
    Ay.set_ydata(my_arduino.Accelerometer['y'])
    Az.set_ydata(my_arduino.Accelerometer['z'])
    
    Gx.set_ydata(my_arduino.Gyroscope['x'])
    Gy.set_ydata(my_arduino.Gyroscope['y'])
    Gz.set_ydata(my_arduino.Gyroscope['z'])
    
    Mx.set_ydata(my_arduino.Magnetometer['x'])
    My.set_ydata(my_arduino.Magnetometer['y'])
    Mz.set_ydata(my_arduino.Magnetometer['z'])

ani = animation.FuncAnimation(fig, animate, interval=200)

Connected     

# 3d Plot Accelerometer-Magnetometer

In [ ]:
import mpl_toolkits.mplot3d.axes3d as p3

my_arduino = arduino()
#Loops the in_background function
try:
    thread = threading.Thread(target = my_arduino.read)
    thread.start() 

except KeyboardInterrupt:
    thread._Thread_stop() 
    #sys.exit()
    

fig = plt.figure()
ax  = p3.Axes3D(fig)

A, = ax.plot([0,my_arduino.Accelerometer['x'][-1]],[0,my_arduino.Accelerometer['y'][-1]],[0,my_arduino.Accelerometer['z'][-1]],'o-',color='r')
M, = ax.plot([0,my_arduino.Magnetometer['x'][-1]/50],[0,my_arduino.Magnetometer['y'][-1]/50],[0,my_arduino.Magnetometer['z'][-1]/50],'o-',color='g')

ax.plot([0,1],[0,0],[0,0],'k--');ax.plot([0,-1],[0,0],[0,0],'k:')
ax.plot([0,0],[0,1],[0,0],'k--');ax.plot([0,0],[0,-1],[0,0],'k:')
ax.plot([0,0],[0,0],[0,1],'k--');ax.plot([0,0],[0,0],[0,-1],'k:')

# Setting the axes properties
ax.set_xlim3d([-1.5, 1.5])
ax.set_xlabel('X')

ax.set_ylim3d([-1.5, 1.5])
ax.set_ylabel('Y')

ax.set_zlim3d([-1.5, 1.5])
ax.set_zlabel('Z')

ax.set_title('3D Test')

def animate(i,Accelerometer,Magnetometer,A,M):
    
    Axydata = np.vstack([[0,Accelerometer['x'][-1]],[0,Accelerometer['y'][-1]]])
    A.set_data(Axydata)
    A.set_3d_properties([0,Accelerometer['z'][-1]])

    Mxydata = np.vstack([[0,Magnetometer['x'][-1]/50],[0,Magnetometer['y'][-1]/50]])
    M.set_data(Mxydata)
    M.set_3d_properties([0,Magnetometer['z'][-1]/50])

    # Creating the Animation object
line_ani = animation.FuncAnimation(fig, animate, fargs=(my_arduino.Accelerometer,my_arduino.Magnetometer,A,M)
                                   )

['183502', '0.05', '-1.00', '0.15', '0.85', '0.98', '0.24', '-6.20', '57.71', '-13.82']]

/var/folders/js/5j9mcztd6ss89xbc4d0py4z40000gn/T/ipykernel_7997/861405049.py:15: MatplotlibDeprecationWarning: Axes3D(fig) adding itself to the figure is deprecated since 3.4. Pass the keyword argument auto_add_to_figure=False and use fig.add_axes(ax) to suppress this warning. The default value of auto_add_to_figure will change to False in mpl3.5 and True values will no longer work in 3.6.  This is consistent with other Axes classes.
  ax  = p3.Axes3D(fig)


['1', '-.558.29', '-12.39']

Exception in thread Thread-13:
Traceback (most recent call last):
  File "/Users/Tommaso/miniconda3/lib/python3.9/threading.py", line 954, in _bootstrap_inner
    self.run()
  File "/Users/Tommaso/miniconda3/lib/python3.9/threading.py", line 892, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/js/5j9mcztd6ss89xbc4d0py4z40000gn/T/ipykernel_7997/2977669404.py", line 92, in read
  File "/var/folders/js/5j9mcztd6ss89xbc4d0py4z40000gn/T/ipykernel_7997/2977669404.py", line 62, in get_new_data
ValueError: too many values to unpack (expected 10)


['349593', '0.06', '-1.00', '0.16', '0.85', '1.16', '0.61', '-5.87', '58.65', '-12.23']']']]0', '-12.48']

Exception in thread Thread-14:
Traceback (most recent call last):
  File "/Users/Tommaso/miniconda3/lib/python3.9/site-packages/serial/serialposix.py", line 575, in read
    buf = os.read(self.fd, size - len(read))
OSError: [Errno 6] Device not configured

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/Tommaso/miniconda3/lib/python3.9/threading.py", line 954, in _bootstrap_inner
    self.run()
  File "/Users/Tommaso/miniconda3/lib/python3.9/threading.py", line 892, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/js/5j9mcztd6ss89xbc4d0py4z40000gn/T/ipykernel_7997/2977669404.py", line 92, in read
  File "/var/folders/js/5j9mcztd6ss89xbc4d0py4z40000gn/T/ipykernel_7997/2977669404.py", line 60, in get_new_data
  File "/var/folders/js/5j9mcztd6ss89xbc4d0py4z40000gn/T/ipykernel_7997/2977669404.py", line 40, in read_serial_monitor
  File "/Users/Tommaso/miniconda3/lib/python3.9/site-packages/seria

# Yaw Pitch Roll

In [13]:
my_arduino = arduino()
#Loops the in_background function
try:
    thread = threading.Thread(target = my_arduino.read)
    thread.start() 

except KeyboardInterrupt:
    thread._Thread_stop() 
    #sys.exit()
    
fig, (ax1,ax2,ax3) = plt.subplots(3,1,figsize=[6.4,4.8*3])

YawMagnetometer,   = ax1.plot(np.arange(1000),my_arduino.Yaw['Magnetometer'])
YawGyroscope,      = ax1.plot(np.arange(1000),my_arduino.Yaw['Gyroscope'])
YawComplementary,  = ax1.plot(np.arange(1000),my_arduino.Yaw['Complementary'])

PitchAccelerometer,= ax2.plot(np.arange(1000),my_arduino.Pitch['Accelerometer'])
PitchGyroscope,    = ax2.plot(np.arange(1000),my_arduino.Pitch['Gyroscope'])
PitchComplementary,= ax2.plot(np.arange(1000),my_arduino.Pitch['Complementary'])

RollAccelerometer, = ax3.plot(np.arange(1000),my_arduino.Roll['Accelerometer'])
RollGyroscope,     = ax3.plot(np.arange(1000),my_arduino.Roll['Gyroscope'])
RollComplementary, = ax3.plot(np.arange(1000),my_arduino.Roll['Complementary'])

ax1.grid(True);ax2.grid(True);ax3.grid(True)

lim = [-np.pi*1.1,np.pi*1.1]
ax1.set_ylim(lim);ax2.set_ylim(lim);ax3.set_ylim(lim)

def animate(i):
    
    YawMagnetometer.set_ydata(my_arduino.Yaw['Magnetometer'])
    YawGyroscope.set_ydata(my_arduino.Yaw['Gyroscope'])
    YawComplementary.set_ydata(my_arduino.Yaw['Complementary'])
    
    PitchAccelerometer.set_ydata(my_arduino.Pitch['Accelerometer'])
    PitchGyroscope.set_ydata(my_arduino.Pitch['Gyroscope'])
    PitchComplementary.set_ydata(my_arduino.Pitch['Complementary'])
    
    RollAccelerometer.set_ydata(my_arduino.Roll['Accelerometer'])
    RollGyroscope.set_ydata(my_arduino.Roll['Gyroscope'])
    RollComplementary.set_ydata(my_arduino.Roll['Complementary'])

ani = animation.FuncAnimation(fig, animate, interval=200)

Connecting...

Exception in thread Thread-10:
Traceback (most recent call last):
  File "/Users/Tommaso/miniconda3/lib/python3.9/threading.py", line 954, in _bootstrap_inner
    self.run()
  File "/Users/Tommaso/miniconda3/lib/python3.9/threading.py", line 892, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/js/5j9mcztd6ss89xbc4d0py4z40000gn/T/ipykernel_28936/610916887.py", line 93, in read
  File "/var/folders/js/5j9mcztd6ss89xbc4d0py4z40000gn/T/ipykernel_28936/610916887.py", line 61, in get_new_data


Connected     

  File "/var/folders/js/5j9mcztd6ss89xbc4d0py4z40000gn/T/ipykernel_28936/610916887.py", line 40, in read_serial_monitor
  File "/Users/Tommaso/miniconda3/lib/python3.9/site-packages/serial/serialposix.py", line 595, in read
    raise SerialException(
serial.serialutil.SerialException: device reports readiness to read but returned no data (device disconnected or multiple access on port?)


In [5]:
import mpl_toolkits.mplot3d.axes3d as p3

my_arduino = arduino()
#Loops the in_background function
try:
    thread = threading.Thread(target = my_arduino.read)
    thread.start() 

except KeyboardInterrupt:
    thread._Thread_stop() 
    #sys.exit()
    

fig = plt.figure()
ax  = p3.Axes3D(fig)

YPR, = ax.plot(my_arduino.Yaw['Complementary'][-1],my_arduino.Pitch['Complementary'][-1],my_arduino.Roll['Complementary'][-1],'o-',color='r')

ax.plot([0,1],[0,0],[0,0],'k--');ax.plot([0,-1],[0,0],[0,0],'k:')
ax.plot([0,0],[0,1],[0,0],'k--');ax.plot([0,0],[0,-1],[0,0],'k:')
ax.plot([0,0],[0,0],[0,1],'k--');ax.plot([0,0],[0,0],[0,-1],'k:')

# Setting the axes properties
ax.set_xlim3d([-1.5, 1.5])
ax.set_xlabel('Yaw')

ax.set_ylim3d([-1.5, 1.5])
ax.set_ylabel('Pitch')

ax.set_zlim3d([-1.5, 1.5])
ax.set_zlabel('Roll')

ax.set_title('3D Test')

def animate(i,my_arduino,YPR):
    
    xydata = np.vstack([my_arduino.Yaw['Complementary'][-1],my_arduino.Pitch['Complementary'][-1]])
    YPR.set_data(xydata)
    YPR.set_3d_properties(my_arduino.Roll['Complementary'][-1])

    # Creating the Animation object
line_ani = animation.FuncAnimation(fig, animate, fargs=(my_arduino,YPR)
                                   )

['2126935', '-0.00', '-0.03', '0.99', '0.92', '0.67', '0.43', '17.16', '14.36', '-24.26']]]]]]]

# 3D SUP

In [6]:
def Rx(theta_x):#roll
    return np.array([[1, 0, 0],[0, np.cos(theta_x),-np.sin(theta_x)],[0, np.sin(theta_x),np.cos(theta_x)]])

def Ry(theta_y):#pitch
    return np.array([[np.cos(theta_y), 0, np.sin(theta_y)],[0, 1, 0],[-np.sin(theta_y), 0, np.cos(theta_y)]])

def Rz(theta_z):#yaw
    return np.array([[np.cos(theta_z),-np.sin(theta_z), 0],[np.sin(theta_z),np.cos(theta_z), 0],[0,0,1]])

def rotation(SUP,yaw,pitch,roll):

    R = np.dot(Rz(yaw),np.dot(Ry(pitch),Rx(roll)))
    
    return np.dot(R,SUP)

SUP = np.array([[-3,3,4,3,-3,-3],[-1,-1,0,1,1,-1],[0,0,0,0,0,0]])

STICKMAN = np.array([[0,0,0,0,0,0,0,0,0,0],[-0.75,0,0.75,0,0,0,-1,-1,0,1],[0,2,0,2,4.5,4,3,3,4,3]])

In [7]:
import mpl_toolkits.mplot3d.axes3d as p3

my_arduino = arduino()
#Loops the in_background function
try:
    thread = threading.Thread(target = my_arduino.read)
    thread.start() 

except KeyboardInterrupt:
    thread._Thread_stop() 
    #sys.exit()
    

fig = plt.figure()
ax  = p3.Axes3D(fig)

SUPline,      = ax.plot(SUP[0],SUP[1],SUP[2])
STICKMANline, = ax.plot(STICKMAN[0],STICKMAN[1],STICKMAN[2])

ax.plot([0,1],[0,0],[0,0],'k--');ax.plot([0,-1],[0,0],[0,0],'k:')
ax.plot([0,0],[0,1],[0,0],'k--');ax.plot([0,0],[0,-1],[0,0],'k:')
ax.plot([0,0],[0,0],[0,1],'k--');ax.plot([0,0],[0,0],[0,-1],'k:')

# Setting the axes properties
ax.set_xlim3d([-10,10])
ax.set_xlabel('X')

ax.set_ylim3d([-10,10])
ax.set_ylabel('Y')

ax.set_zlim3d([-10,10])
ax.set_zlabel('Z')

ax.set_title('3D Test')

def animate(i):
    
    rotatedSUP      = rotation(SUP,my_arduino.Yaw['Complementary'][-1],my_arduino.Pitch['Complementary'][-1],my_arduino.Roll['Complementary'][-1])
    rotatedSTICKMAN = rotation(STICKMAN,my_arduino.Yaw['Complementary'][-1],my_arduino.Pitch['Complementary'][-1],my_arduino.Roll['Complementary'][-1])
    
    SUPline.set_data(rotatedSUP[:2])
    SUPline.set_3d_properties(rotatedSUP[2])
    
    STICKMANline.set_data(rotatedSTICKMAN[:2])
    STICKMANline.set_3d_properties(rotatedSTICKMAN[2])
    
ani = animation.FuncAnimation(fig, animate, interval=200)

Exception in thread Thread-7:
Traceback (most recent call last):
  File "/Users/Tommaso/miniconda3/lib/python3.9/threading.py", line 954, in _bootstrap_inner
/var/folders/js/5j9mcztd6ss89xbc4d0py4z40000gn/T/ipykernel_8848/167336157.py:15: MatplotlibDeprecationWarning: Axes3D(fig) adding itself to the figure is deprecated since 3.4. Pass the keyword argument auto_add_to_figure=False and use fig.add_axes(ax) to suppress this warning. The default value of auto_add_to_figure will change to False in mpl3.5 and True values will no longer work in 3.6.  This is consistent with other Axes classes.
  ax  = p3.Axes3D(fig)
    self.run()


Connected     

  File "/Users/Tommaso/miniconda3/lib/python3.9/threading.py", line 892, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/js/5j9mcztd6ss89xbc4d0py4z40000gn/T/ipykernel_8848/610916887.py", line 93, in read
  File "/var/folders/js/5j9mcztd6ss89xbc4d0py4z40000gn/T/ipykernel_8848/610916887.py", line 61, in get_new_data


  File "/var/folders/js/5j9mcztd6ss89xbc4d0py4z40000gn/T/ipykernel_8848/610916887.py", line 40, in read_serial_monitor
  File "/Users/Tommaso/miniconda3/lib/python3.9/site-packages/serial/serialposix.py", line 595, in read
    raise SerialException(
serial.serialutil.SerialException: device reports readiness to read but returned no data (device disconnected or multiple access on port?)
